In [1]:
from transformers import AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer 
import numpy as np
from datasets import load_metric
from datasets import Dataset
from datasets import load_dataset

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [20]:
#load dataset and refactor columns

def saveTrainAndTest(train_texts, test_texts): 
    train_texts = train_texts.reset_index(drop = True) 
    test_texts = test_texts.reset_index(drop = True)
    
    train_texts.to_csv("/content/gdrive/My Drive/refactored_package2/trainData.csv", index = False)
    test_texts.to_csv("/content/gdrive/My Drive/refactored_package2/testData.csv", index=False)

dataset = pd.read_csv("/content/gdrive/My Drive/refactored_package2/tar_train_data.csv")

dataset["label"] = pd.Categorical(dataset["label"])

dataset["label"] = dataset["label"].cat.codes

dataset.insert(0, "text", dataset.pop("sentence"))

dataset.pop("source")

train_texts, test_texts = train_test_split(dataset, train_size=0.8, random_state=42)

saveTrainAndTest(train_texts, test_texts)

In [27]:
raw_datasets = load_dataset('csv', data_files={'train': "trainData.csv",
                                              'test': 'testData.csv'})
print(dataset)

def tokenize_function(raw_dataset):
    return tokenizer(raw_dataset["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
test_dataset = tokenized_datasets["test"].shuffle(seed=42)

Using custom data configuration default-daba1b87c3f54bc6
Reusing dataset csv (/Users/vedantmathur/.cache/huggingface/datasets/csv/default-daba1b87c3f54bc6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/vedantmathur/.cache/huggingface/datasets/csv/default-daba1b87c3f54bc6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-19d724e98b692fe2.arrow
Loading cached processed dataset at /Users/vedantmathur/.cache/huggingface/datasets/csv/default-daba1b87c3f54bc6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-60e951dbaf176c90.arrow
Loading cached shuffled indices for dataset at /Users/vedantmathur/.cache/huggingface/datasets/csv/default-daba1b87c3f54bc6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-be4ea3a059965a8a.arrow
Loading cached shuffled indices for dataset at /Users/vedantmathur/.cache/huggingface/datasets/csv/default-daba1b87c3f54bc6

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 704
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 177
    })
})


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

training_args = TrainingArguments("test_trainer")


model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,     
    eval_dataset=test_dataset       
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/vedantmathur/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cla

Step,Training Loss


In [12]:
print(len(test_texts))

177


In [ ]:
import torch 

class EarthquakeDataset(torch.utils.data.Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer 
        self.max_len = max_len
    

    def __getitem__(self, index):
        print(self.sentences)
        sentence = str(self.sentences[index])
        label = self.labels[index]
        
        encoding = self.tokenizer.encode_plus(
      sentence,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')
        
        return {
      'sentence': sentence,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'label': torch.tensor(label, dtype=torch.long)
    }
    

    def __len__(self):
        return len(self.labels)    
    
train_dataset = EarthquakeDataset(train_texts["sentences"], train_labels, tokenizer, 500) 
test_dataset = EarthquakeDataset(test_texts["sentences"], test_labels, tokenizer, 500)

In [ ]:
testimdb = load_dataset("imdb")
print(testimdb)

In [3]:
model = BertForSequenceClassification.from_pretrained("fineTunedBert")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [4]:
def tokenize_sentence(sentence):
    return tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt")

In [43]:
from datasets import Dataset
sentenceFrame = pd.read_csv("sentences.csv")
sentenceFrame = (sentenceFrame.drop("Unnamed: 0", axis = 1))
sentences = (sentenceFrame.to_numpy())


tokenizedSentences = tokenize_sentence([sentences[x][0] for x in range(0, len(sentences))])

class EarthquakeDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}
    

test_dataset = EarthquakeDataset(tokenizedSentences)


In [ ]:
trainer = Trainer(model=model)
print(trainer.predict(test_dataset))

***** Running Prediction *****
  Num examples = 654
  Batch size = 8


In [ ]:
for x in range(0, len(tokenizedSentences)): 
    print(model(**tokenizedSentences[x]).logits)

tensor([[-1.5454, -0.1363, -1.4517,  3.8537]], grad_fn=<AddmmBackward>)
tensor([[-2.1535, -0.5539,  3.9677, -1.2809]], grad_fn=<AddmmBackward>)
tensor([[-1.1131, -0.1695,  0.3622,  1.1315]], grad_fn=<AddmmBackward>)


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

text = "Reuters reporting that there are several damaged buildings in Acapulco. There have already been several aftershocks as well. Reuters reporting that there are several damaged buildings in Acapulco. There have already been several aftershocks as well. Reuters reporting that there are several damaged buildings in Acapulco. There have already been several aftershocks as well. Reuters reporting that there are several damaged buildings in Acapulco. There have already been several aftershocks as well."
t5_prepared_Text = "summarize: "+text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")
# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment.


In [ ]:
pip install s